# CNN for image classification analysis
In this notebook a CNN model will be developed to classify bark according to tree species. The CNN will be trained to classify the original images. After the training the CNN will be evaluated with some post-hoc model analysis methods like LIME and SHAP.

## Organizing the data structure (only done once, after downloading the dataset)

First, the dataset (https://www.kaggle.com/datasets/saurabhshahane/barkvn50) should be downloaded to directory: "./data/BarkVN-50/" and unzipped. You should then have the following structure:
- data
    - BarkVN-50
        - BarkVN-50_mendeley
            - Acacia
            - Adenanthera microsperma
            - Adenieum species
            - Anacardium occidentale
            - ...

Since this is not ideal for this CNN, a subset of the data is selected and split into training data using the code in the next cell:

In [1]:
# import helpers.split
# helpers.split.train_test_split()

Note: this cell only needs to be executed once (this is why it is commented out by default).

After execution the new data structure looks like this:
- data
    - BarkVN-50
        - BarkVN-50_mendeley
            - Acacia
            - Adenanthera microsperma
            - Adenieum species
            - Anacardium occidentale
            - ...
        - Test
            - Adenanthera microsperma
            - Cananga odorata
            - Cedrus
            - Cocos nucifera
            - Dalvergia oliveri
        - Train
            - Adenanthera microsperma
            - Cananga odorata
            - Cedrus
            - Cocos nucifera
            - Dalvergia oliveri

Note: the directory "./data/BarkVN-50/BarkVN-50_mendeley" may be deleted after this step.

## Loading the Dataset and creating DataLoaders
Since the used dataset is a custom one, we need to first create a custom Dataset for loading, transforming and delivering datapoints.

In [2]:
from helpers.dataset import BarkVN50Dataset
from torch.utils.data import DataLoader
from torch import device
from torch.cuda import is_available

# recognizing device
DEVICE = device("cuda" if is_available() else "cpu")

# load train and test dataset
train_dataset = BarkVN50Dataset(train=True, device=DEVICE)
test_dataset = BarkVN50Dataset(train=False, device=DEVICE)

# create DataLoaders that automatically create minibatches and shuffle the data
train_dataloader = DataLoader(train_dataset, batch_size=39, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=39, shuffle=True)

## Training the CNN model

### Initialization
Now that the data is ready to be used, we can load the CNN model:

In [3]:
from helpers.cnn import ConvolutionalNeuralNetwork

model = ConvolutionalNeuralNetwork()
model.to(device=DEVICE)

ConvolutionalNeuralNetwork(
  (cnn): Sequential(
    (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (categorizer): Sequential(
    (0): Linear(in_features=60600, out_features=4800, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4800, out_features=5, bias=True)
  )
)

Initialize the hyperaparameters, optimizer and the criterion (loss function):

In [4]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss

# hyperparameters
num_epochs = 10
learning_rate = 3e-5

# optimizer and loss function
model.train()
optimizer = Adam(model.parameters(), lr=learning_rate)
criterion = CrossEntropyLoss()

### Training the CNN model
And finally train the model:

In [5]:
from helpers.train import train_cnn

train_cnn(
    num_epochs=num_epochs,
    model=model,
    criterion=criterion,
    dataloader=train_dataloader,
    optimizer=optimizer,
)

Epoch [1/10], Step [1/11], Loss: 7.6943
Epoch [1/10], Step [2/11], Loss: 104.4559
Epoch [1/10], Step [3/11], Loss: 191.0229
Epoch [1/10], Step [4/11], Loss: 305.4359
Epoch [1/10], Step [5/11], Loss: 252.1283
Epoch [1/10], Step [6/11], Loss: 168.2683
Epoch [1/10], Step [7/11], Loss: 138.5073
Epoch [1/10], Step [8/11], Loss: 163.5892
Epoch [1/10], Step [9/11], Loss: 97.1250
Epoch [1/10], Step [10/11], Loss: 97.0325
Epoch [1/10], Step [11/11], Loss: 86.0170
Epoch [2/10], Step [1/11], Loss: 80.2032
Epoch [2/10], Step [2/11], Loss: 43.2833
Epoch [2/10], Step [3/11], Loss: 28.1741
Epoch [2/10], Step [4/11], Loss: 33.0421
Epoch [2/10], Step [5/11], Loss: 29.8745
Epoch [2/10], Step [6/11], Loss: 14.1201
Epoch [2/10], Step [7/11], Loss: 15.5952
Epoch [2/10], Step [8/11], Loss: 27.9101
Epoch [2/10], Step [9/11], Loss: 26.9730
Epoch [2/10], Step [10/11], Loss: 31.8553
Epoch [2/10], Step [11/11], Loss: 30.2992
Epoch [3/10], Step [1/11], Loss: 35.8994
Epoch [3/10], Step [2/11], Loss: 10.8097
Epoch 

If the model should be serializable and deserializable, it can be saved:

In [6]:
from torch import save
from datetime import datetime

time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
save(model.state_dict(), f"models/{time}.pt")